In [1]:
#We need to first clean and concatenate the datasets from 2019
#Import libararies 
import pandas as pd
import numpy as np
import glob

#Load the 2019 datasets
df1=pd.read_csv("./datasets_2019/20190103-divvy-tripdata.csv")
df2=pd.read_csv("./datasets_2019/20190406-divvy-tripdata.csv")
df3=pd.read_csv("./datasets_2019/20190709-divvy-tripdata.csv")
df4=pd.read_csv("./datasets_2019/20191012-divvy-tripdata.csv")


# Rename the colum names of df2 to match the rest
df2 = df2.rename(columns={'01 - Rental Details Rental ID': 'trip_id', '01 - Rental Details Local Start Time': 'start_time',
                          '01 - Rental Details Local End Time':'end_time','01 - Rental Details Bike ID': 'bikeid',
                          '01 - Rental Details Duration In Seconds Uncapped':'tripduration', '03 - Rental Start Station ID':'from_station_id', 
                          '03 - Rental Start Station Name':'from_station_name', '02 - Rental End Station ID':'to_station_id',
                          '02 - Rental End Station Name':'to_station_name', 'User Type':'usertype','Member Gender':'gender',
                          '05 - Member Details Member Birthday Year':'birthyear'
                          })

# Concatenate all the datasets
combined_df_2019= pd.concat([df1, df2, df3, df4], ignore_index=True)


#Change the values in column user type (Subscriber to member and customer to casual)

combined_df_2019['usertype']= combined_df_2019['usertype'].replace({'Subscriber': 'member',
    'Customer': 'casual'})

# Rename columns to match the datasets from 2020 to 2024
combined_df_2019 = combined_df_2019.rename(columns={
    'trip_id': 'ride_id',
    'start_time': 'started_at',
    'end_time': 'ended_at',
    'from_station_name': 'start_station_name',
    'from_station_id': 'start_station_id',
    'to_station_name': 'end_station_name',
    'to_station_id': 'end_station_id',
    'usertype': 'member_casual'
})

# Add missing columns with empty or NaN values to match the column names of the datasest from 2020 to 2024
combined_df_2019['rideable_type'] = np.nan
combined_df_2019['start_lat'] = np.nan
combined_df_2019['start_lng'] = np.nan
combined_df_2019['end_lat'] = np.nan
combined_df_2019['end_lng'] = np.nan

# Drop columns that don't exist in the datasest from 2020 to 2024
combined_df_2019= combined_df_2019.drop(['bikeid', 'tripduration', 'gender', 'birthyear'], axis=1)


 # Reorder columns to match the datasets from 2020 to 2024
desired_order = [
    'ride_id', 'rideable_type', 'started_at', 'ended_at',
    'start_station_name', 'start_station_id', 'end_station_name', 'end_station_id',
    'start_lat', 'start_lng', 'end_lat', 'end_lng', 'member_casual'
]

combined_df_2019  = combined_df_2019[desired_order]

#Save the cleaned dataset to CSV
combined_df_2019.to_csv('datasets_2019_2024/2019-divvy-tripdata_cleaned.csv', index=False)
print("Successfully saved the cleaned dataset. Combined dataset shape:", combined_df_2019.shape)

Successfully saved the cleaned dataset. Combined dataset shape: (3818004, 13)


In [3]:
# Here we access all the datasets and try to concatenate them
# Path to datasets folder
folder_path = "./datasets_2019_2024/"

#Pattern to match dataset for 2019
file_2019 = folder_path +"2019-divvy-tripdata_cleaned.csv"
df_2019 = pd.read_csv(file_2019)

#Pattern to match all CSV files for year 2020 to 2024
file_pattern = folder_path + "202*-divvy-tripdata.csv"

#Get list of matching files
all_files = glob.glob(file_pattern)

#List to hold dataframes
df_list= [df_2019]

for file in all_files:
    df = pd.read_csv(file)
    df_list.append(df)

# Combine all datasets into one dataframe
combined_df = pd.concat(df_list, ignore_index= True)
print("Combined dataset shape before cleaning:", combined_df.shape)

# Filter invalid coordinates
combined_df = combined_df[(combined_df['start_lat'].between(41.5, 42.2) | combined_df['start_lat'].isna()) &
    (combined_df['start_lng'].between(-88.1, -87) | combined_df['start_lng'].isna()) &
    (combined_df['end_lat'].between(41.5, 42.2) | combined_df['end_lat'].isna()) &
    (combined_df['end_lng'].between(-88.1, -87) | combined_df['end_lng'].isna())]

#Fill rideable type's missing values with unknown
combined_df['rideable_type'] = combined_df['rideable_type'].fillna('unknown')

#Fill the missing stations names with unknown
combined_df['start_station_name'] = combined_df['start_station_name'].fillna('Unknown')
combined_df['end_station_name'] = combined_df['end_station_name'].fillna('Unknown')
combined_df['start_station_id'] = combined_df['start_station_id'].fillna('Unknown')
combined_df['end_station_id'] = combined_df['end_station_id'].fillna('Unknown')

#Split the dataset to convert started_at and ended_at columns to datetime type correctly
combined_df_part1 = combined_df.iloc[:26036568].copy()
combined_df_part2 = combined_df.iloc[26036568:].copy()

# First part — no microseconds-convert it to datetime
combined_df_part1['started_at'] = pd.to_datetime(combined_df_part1['started_at'], errors='coerce')
combined_df_part1['ended_at'] = pd.to_datetime(combined_df_part1['ended_at'], errors='coerce')

# Second part — has microseconds-convert to datetime with the explicit parse method to parse millisecond
combined_df_part2['started_at'] = pd.to_datetime(combined_df_part2['started_at'], format='%Y-%m-%d %H:%M:%S.%f', errors='coerce')
combined_df_part2['ended_at'] = pd.to_datetime(combined_df_part2['ended_at'], format='%Y-%m-%d %H:%M:%S.%f', errors='coerce')

#Conctanate the dfs
df_cleaned = pd.concat([combined_df_part1, combined_df_part2], ignore_index=True)

#Create the neccessary columns for performing aggregations
df_cleaned['year'] = df_cleaned['started_at'].dt.year
df_cleaned['month'] = df_cleaned['started_at'].dt.to_period('M').astype(str)  # e.g., '2024-01'
df_cleaned['month_number'] = df_cleaned['started_at'].dt.month
df_cleaned['month_name'] = df_cleaned['started_at'].dt.strftime('%B')         # e.g., 'January'
df_cleaned['day_of_week'] = df_cleaned['started_at'].dt.day_name()
df_cleaned['hour'] = df_cleaned['started_at'].dt.hour
df_cleaned['ride_duration_min'] = ((df_cleaned['ended_at'] - df_cleaned['started_at']).dt.total_seconds() / 60).round(2) # round(2)

#Save cleaned dataset
df_cleaned.to_csv('cleaned_dataset_2019_2024/divvy_2019_2024_cleaned.csv',index=False)
print("Exported cleaned dataset: divvy_cleaned_2019_2024.csv . Cleaned datase shape after cleaning:", df_cleaned.shape)

Combined dataset shape before cleaning: (30202912, 13)
Exported cleaned dataset: divvy_cleaned_2019_2024.csv . Cleaned datase shape after cleaning: (30202834, 20)


In [5]:
#Load the cleaned final csv for perfroming the aggregation to avoid running the whole cleaning code every time

df_cleaned= pd.read_csv('cleaned_dataset_2019_2024/divvy_2019_2024_cleaned.csv', low_memory= False)
print('Final cleaned CSV loaded for performing the aggregations.')

Final cleaned CSV loaded for performing the aggregations.


In [7]:
#KPI Overview - rider per year by user type
agg_overview =(df_cleaned.groupby(['year', 'member_casual']).agg(number_of_rides =('ride_id', 'count')).reset_index())

# Total rides and growth by year
agg_total_year = (df_cleaned.groupby('year').agg(total_rides=('ride_id','count')).reset_index())
agg_total_year['growth_%'] = (agg_total_year['total_rides'].pct_change()* 100).round(2)

#Monthly Ride Trends
agg_monthly = (df_cleaned.groupby(['year', 'month', 'month_number', 'month_name', 'member_casual']).agg(number_of_rides=('ride_id', 'count'))
               .reset_index().sort_values(['year', 'month_number']))

#Ride duration by user and bike type
agg_duration=(df_cleaned.groupby(['year','member_casual', 'rideable_type']).agg(avg_duration_min=('ride_duration_min','mean')).reset_index())

#Ride Type Usage
agg_ride_type=(df_cleaned.groupby(['year','member_casual','rideable_type']).agg(number_of_rides=('ride_id','count')).reset_index())

#Weekday annd hour usage
agg_day_hour= (df_cleaned.groupby(['day_of_week','hour']).agg(number_of_rides=('ride_id','count')).reset_index())


#Top start and stations by year

df_filtered = df_cleaned[
    (df_cleaned['start_station_name'].notna()) &
    (df_cleaned['end_station_name'].notna()) &
    (df_cleaned['start_station_name'] != 'Unknown') &
    (df_cleaned['end_station_name'] != 'Unknown')
].copy()

agg_start_station = (
    df_filtered.groupby(['start_station_name', 'year'])
    .agg(number_of_rides=('ride_id', 'count'))
    .reset_index()
    .sort_values(['year', 'number_of_rides'], ascending=[True, False])
    )
agg_end_station = (
    df_filtered.groupby(['end_station_name', 'year'])
    .agg(number_of_rides=('ride_id', 'count'))
    .reset_index()
    .sort_values(['year', 'number_of_rides'], ascending=[True, False])
    )

#Top routes (start to end station pairs)
# Create the route column
df_filtered.loc[:, 'route'] = df_filtered['start_station_name'] + ' → ' + df_filtered['end_station_name']

# Aggregate top 10 routes per year
agg_routes = (
    df_filtered.groupby(['route', 'year', 'member_casual'])
    .agg(number_of_rides=('ride_id', 'count'))
    .reset_index()
    .sort_values(['year', 'member_casual', 'number_of_rides'], ascending=[True, True, False])
    .groupby(['year', 'member_casual'])
    .head(10)
    .reset_index(drop=True)
)

#Ride Duration Segmentation
# Create the duration buckets (adjusted for df_cleaned)
bins = [0, 5, 10, 20, 30, 60, 120, 240, 1440]
labels = ['<5 min', '5-10', '10-20', '20-30', '30-60', '1-2 hr', '2-4 hr', '>4 hr']

df_cleaned['duration_bucket'] = pd.cut(df_cleaned['ride_duration_min'], bins=bins, labels=labels, right=False)

# Group by duration_bucket and member_casual, count rides
agg_duration_member = (
    df_cleaned.groupby(['duration_bucket','rideable_type','year', 'member_casual'],observed=True)
    .agg(number_of_rides=('ride_id', 'count'))
    .reset_index()
)

# Load Chicago Zip cod csv file for coordinate to ZIP Code Mapping
zip_df = pd.read_csv("data/chicago_zip_codes.csv", dtype={'ZIP_Code': str}) 

#Import Library
from sklearn.neighbors import BallTree

# Convert to radians for BallTree
zip_coords = np.radians(zip_df[['Latitude', 'Longitude']]) 

#Build BallTree for fast nearest neigbor search
tree = BallTree(zip_coords, metric='haversine')

#Assign ZIP to start coordinates
# Prepare start coords and convert to radians
start_coords = df_cleaned[['start_lat', 'start_lng']].dropna()
start_coords_rad = np.radians(start_coords)

# Query nearest ZIP
distances_start, indices_start = tree.query(start_coords_rad, k=1)

# Map indices to ZIP codes
start_zip_matches = zip_df.iloc[indices_start.flatten()]['ZIP_Code'].values

# Assign ZIPs to df_cleaned
df_cleaned['start_zip'] = np.nan
df_cleaned['start_zip'] = df_cleaned['start_zip'].astype(object)
df_cleaned.loc[start_coords.index, 'start_zip'] = start_zip_matches

#Assign ZIP to end coordinates
# Prepare end coords and convert to radians
end_coords = df_cleaned[['end_lat', 'end_lng']].dropna()
end_coords_rad = np.radians(end_coords)

# Query nearest ZIP
distances_end, indices_end = tree.query(end_coords_rad, k=1)

# Map indices to ZIP codes
end_zip_matches = zip_df.iloc[indices_end.flatten()]['ZIP_Code'].values

# Assign ZIPs to df_cleaned
df_cleaned['end_zip'] = np.nan
df_cleaned['end_zip'] = df_cleaned['end_zip'].astype(object)
df_cleaned.loc[end_coords.index, 'end_zip'] = end_zip_matches

# Fill missing ZIPs with 'Unknown'
df_cleaned['start_zip'] = df_cleaned['start_zip'].fillna('Unknown')
df_cleaned['end_zip'] = df_cleaned['end_zip'].fillna('Unknown')

#Geographic insights: popular ZIP routes
agg_zip_routes = (
    df_cleaned
    .groupby(['start_zip', 'end_zip', 'year'])
    .agg(number_of_rides=('ride_id', 'count'),
         avg_ride_duration_min=('ride_duration_min', 'mean'))
    .reset_index()
)
agg_zip_routes['avg_ride_duration_min'] = agg_zip_routes['avg_ride_duration_min'].round(2)

#Cleaning the ZIP formats
agg_zip_routes['start_zip'] = agg_zip_routes['start_zip'].astype(str).str.replace(r'\.0$', '', regex=True).str.zfill(5)
agg_zip_routes['end_zip'] = agg_zip_routes['end_zip'].astype(str).str.replace(r'\.0$', '', regex=True).str.zfill(5)

#Removing the unknows from the aggregate table
agg_zip_routes = agg_zip_routes[
    (agg_zip_routes['start_zip'] != 'Unknown') &
    (agg_zip_routes['end_zip'] != 'Unknown')
]

#Make sure the ZIP codes match the pattern of 5 digits
agg_zip_routes = agg_zip_routes[
    agg_zip_routes['start_zip'].str.match(r'^\d{5}$') &
    agg_zip_routes['end_zip'].str.match(r'^\d{5}$')
]

#Save aggregations to csv
agg_overview.to_csv("aggregations/agg_overview.csv", index=False)
print("Saved: aggregations/agg_overview.csv")

agg_total_year.to_csv("aggregations/agg_total_year.csv", index=False)
print("Saved: aggregations/agg_total_year.csv")

agg_monthly.to_csv("aggregations/agg_monthly.csv", index=False)
print("Saved: aggregations/agg_monthly.csv")

agg_duration.to_csv("aggregations/agg_duration.csv", index=False)
print("Saved: aggregations/agg_duration.csv")

agg_ride_type.to_csv("aggregations/agg_ride_type.csv", index=False)
print("Saved: aggregations/agg_ride_type.csv")

agg_day_hour.to_csv("aggregations/agg_day_hour.csv", index=False)
print("Saved: aggregations/agg_day_hour.csv")

agg_start_station.to_csv("aggregations/agg_start_station.csv", index=False)
print("Saved: aggregations/agg_start_station.csv")

agg_end_station.to_csv("aggregations/agg_end_station.csv", index=False)
print("Saved: aggregations/agg_end_station.csv")

agg_routes.to_csv("aggregations/agg_routes.csv", index=False)
print("Saved: aggregations/agg_routes.csv")

agg_duration_member.to_csv("aggregations/agg_duration_member.csv", index=False)
print("Saved: aggregations/agg_duration_member.csv")

agg_zip_routes.to_csv("aggregations/agg_zip_routes.csv", index=False)
print("Saved: aggregations/agg_zip_routes.csv")
print('All aggrepagtions have been saved to the folder aggregations.')

Saved: aggregations/agg_overview.csv
Saved: aggregations/agg_total_year.csv
Saved: aggregations/agg_monthly.csv
Saved: aggregations/agg_duration.csv
Saved: aggregations/agg_ride_type.csv
Saved: aggregations/agg_day_hour.csv
Saved: aggregations/agg_start_station.csv
Saved: aggregations/agg_end_station.csv
Saved: aggregations/agg_routes.csv
Saved: aggregations/agg_duration_member.csv
Saved: aggregations/agg_zip_routes.csv
All aggrepagtions have been saved to the folder aggregations.
